# Loading data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation 
from pyts.image import MarkovTransitionField
import dask.dataframe as dd  
import pandas as pd
    

from scipy import signal as sign
from PIL import Image
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import matthews_corrcoef

import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

spectrogram = 'spectrogram' # 'spectrogram', 'mel', 'mfcc', 'mtf'

# # root_dir = "/home/polivares/scratch/Datasets/PowerLineFaults/"
root_dir = '/home/polivares/Dropbox/Work/PostDoc/PowerLineFaults/'

X_full = np.load(f"{root_dir}dataset_{spectrogram}/full/images_full.npy")
y_full = np.load(f"{root_dir}dataset_{spectrogram}/full/labels_full.npy").reshape(-1)


np.random.seed(42)
tf.random.set_seed(42)

# Index 1

index_1 = np.where(y_full==1)[0]
len_index_1 = len(index_1)
index_train_1, index_val_1, index_test_1 = index_1[:len_index_1//3], index_1[len_index_1//3:2*len_index_1//3], index_1[2*len_index_1//3:]

# Index 0

index_0 = np.where(y_full==0)[0]
index_train_0, index_val_0, index_test_0 = index_0[:len_index_1//3], index_0[len_index_1//3:2*len_index_1//3], index_0[2*len_index_1//3:]

index_train = np.concatenate([index_train_0, index_train_1])
print(index_train)
np.random.shuffle(index_train)

print(index_train)

index_val = np.concatenate([index_val_0, index_val_1])
np.random.shuffle(index_val)

index_test = np.concatenate([index_test_0, index_test_1])
np.random.shuffle(index_test)


2022-10-22 22:39:46.065579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 22:39:46.241274: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-22 22:39:46.285287: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 22:39:46.948159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

[   0    1    2    6    7    8    9   10   11   12   13   14   15   16
   17   18   19   20   21   22   23   24   25   26   27   28   29   30
   31   32   33   34   35   36   37   38   39   40   41   42   43   44
   45   46   47   48   49   50   51   52   53   54   55   56   57   58
   59   60   61   62   63   64   65   66   67   68   69   70   71   72
   73   74   75   76   77   78   79   80   81   82   83   84   85   86
   87   88   89   90   91   92   93   94   95   96   97   98   99  100
  101  102  103  104  105  106  107  108  109  110  111  112  113  114
  115  116  117  118  119  120  121  122  123  124  125  126  127  128
  129  130  131  132  133  134  135  136  137  138  139  140  141  142
  143  144  145  146  147  148  149  150  151  152  153  154  155  156
  157  158  159  160  161  162  163  164  165  166  167  168  169  170
  171  172  173  174  175  176  177    3    4    5  201  202  228  229
  230  270  271  272  279  280  281  285  286  287  290  294  295  296
  380 

In [3]:
def normImages(X):
    for i, image in enumerate(X):
        max_n = np.max(image)
        image /= max_n
        X[i] = image
    return X

X_full = normImages(X_full)

X_train = X_full[index_train]
y_train = y_full[index_train]
X_val = X_full[index_val]
y_val = y_full[index_val]
X_test = X_full[index_test]
y_test = y_full[index_test]

del X_full, y_full

In [4]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

spectrogram = 'mel' # 'spectrogram', 'mel', 'mfcc', 'mtf'

# # root_dir = "/home/polivares/scratch/Datasets/PowerLineFaults/"
root_dir = '/home/polivares/Dropbox/Work/PostDoc/PowerLineFaults/'

X_full_2 = np.load(f"{root_dir}dataset_{spectrogram}/full/images_full.npy")
# y_full_2 = np.load(f"{root_dir}dataset_{spectrogram}/full/labels_full.npy").reshape(-1)

In [5]:
X_full_2 = normImages(X_full_2)

X_train_2 = X_full_2[index_train]
X_val_2 = X_full_2[index_val]
X_test_2 = X_full_2[index_test]

del X_full_2

In [6]:
X_train_c = np.concatenate((X_train, X_train_2), axis=3)
X_val_c = np.concatenate((X_val, X_val_2), axis=3)
X_test_c = np.concatenate((X_test, X_test_2), axis=3)
del X_train, X_train_2, X_val, X_val_2, X_test, X_test_2
# X_test_c = np.concatenate((X_test, X_test_2), axis=3)

# CNN-model

In [ ]:
# Creación de modelo
model = models.Sequential()
# Capas encargadas de obtener información de la imagen
model.add(layers.Conv2D(256, (3, 3), activation='tanh', input_shape=(256,256,2)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
# Capas para la clasificación en base a la información obtenida en 
# capas anteriores
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience = 7)

checkpoint_path='CNN-model_mtf.ckpt'
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.compile(loss = 'bce', optimizer = 'adam', metrics=['accuracy', tf.keras.metrics.Recall()])
history_cnn = model.fit(X_train_c, y_train, epochs=100, 
                        validation_data=(X_val_c, y_val),
                        batch_size=10,
                        callbacks=[early_stopping, cp_callback])

In [ ]:
plt.plot(history_cnn.history['accuracy'], label='accuracy')
plt.plot(history_cnn.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(history_cnn.history['loss'], label='loss')
plt.plot(history_cnn.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
val_loss, val_acc, val_recall = model.evaluate(X_val,  y_val, verbose=2, batch_size=4)

In [ ]:
val_predictions = (model.predict(X_val, batch_size=4)>0.5).reshape(1,-1)[0].astype(int)
val_real_values = y_val.T[0]

In [ ]:
matthews_corrcoef(val_real_values, val_predictions)

In [ ]:
test_loss, test_acc, test_recall = model.evaluate(X_test,  y_test, verbose=2, batch_size=4)

In [ ]:
test_predictions = (model.predict(X_test, batch_size=4)>0.5).reshape(1,-1)[0].astype(int)
test_real_values = y_test.T[0]

In [ ]:
matthews_corrcoef(test_real_values, test_predictions)

# CNN 2

In [ ]:
# # Creación de modelo
# model = models.Sequential()
# # Capas encargadas de obtener información de la imagen
# model.add(layers.Conv2D(256, (3, 3), activation='selu', input_shape=(256,256,1),kernel_initializer='random_normal',))
# model.add(layers.MaxPooling2D((2, 2)))
# # model.add(layers.Dropout(0.3))
# model.add(layers.Conv2D(128, (3, 3), activation='selu',kernel_initializer='random_normal',))
# model.add(layers.MaxPooling2D((2, 2)))
# # model.add(layers.Dropout(0.3))
# model.add(layers.Conv2D(64, (3, 3), activation='selu',kernel_initializer='random_normal',))
# # Capas para la clasificación en base a la información obtenida en 
# # capas anteriores
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='selu',kernel_initializer='random_normal',))
# model.add(layers.Dense(1, activation = 'sigmoid',kernel_initializer='random_normal',))
# model.summary()

In [7]:
# Creación de modelo
# Armado de nuestra red neuronal
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), input_shape=(256, 256, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 252, 252, 32)      1632      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 61, 61, 64)        0

2022-10-22 22:39:56.689759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:39:56.803867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:39:56.804012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:39:56.805399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [8]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience = 10, restore_best_weights=True)

model.compile(loss='binary_crossentropy',
# optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
metrics=METRICS)

In [9]:
history = model.fit(X_train_c, y_train,
batch_size=10,
epochs=100,
verbose=1,
validation_data=(X_val_c, y_val),
callbacks=[early_stopping]
)

Epoch 1/100


2022-10-22 22:36:01.736208: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8300
2022-10-22 22:36:03.034924: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


35/35 [==============================] - 6s 52ms/step - loss: 1.0522 - tp: 116.0000 - fp: 66.0000 - tn: 109.0000 - fn: 59.0000 - accuracy: 0.6429 - precision: 0.6374 - recall: 0.6629 - auc: 0.7052 - prc: 0.7102 - val_loss: 0.4998 - val_tp: 140.0000 - val_fp: 36.0000 - val_tn: 139.0000 - val_fn: 35.0000 - val_accuracy: 0.7971 - val_precision: 0.7955 - val_recall: 0.8000 - val_auc: 0.8436 - val_prc: 0.8042
Epoch 2/100
35/35 [==============================] - 1s 37ms/step - loss: 0.5197 - tp: 130.0000 - fp: 48.0000 - tn: 127.0000 - fn: 45.0000 - accuracy: 0.7343 - precision: 0.7303 - recall: 0.7429 - auc: 0.8184 - prc: 0.8111 - val_loss: 0.4832 - val_tp: 154.0000 - val_fp: 48.0000 - val_tn: 127.0000 - val_fn: 21.0000 - val_accuracy: 0.8029 - val_precision: 0.7624 - val_recall: 0.8800 - val_auc: 0.8678 - val_prc: 0.8410
Epoch 3/100
35/35 [==============================] - 1s 40ms/step - loss: 0.4927 - tp: 144.0000 - fp: 49.0000 - tn: 126.0000 - fn: 31.0000 - accuracy: 0.7714 - precision: 0

In [9]:
history = model.fit(X_train_c, y_train,
batch_size=10,
epochs=100,
verbose=1,
validation_data=(X_val_c, y_val),
callbacks=[early_stopping]
)

Epoch 1/100


2022-10-22 22:40:01.130689: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8300
2022-10-22 22:40:02.411987: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


35/35 [==============================] - 5s 54ms/step - loss: 0.9750 - tp: 129.0000 - fp: 59.0000 - tn: 116.0000 - fn: 46.0000 - accuracy: 0.7000 - precision: 0.6862 - recall: 0.7371 - auc: 0.7375 - prc: 0.7189 - val_loss: 0.5282 - val_tp: 138.0000 - val_fp: 37.0000 - val_tn: 138.0000 - val_fn: 37.0000 - val_accuracy: 0.7886 - val_precision: 0.7886 - val_recall: 0.7886 - val_auc: 0.8524 - val_prc: 0.8174
Epoch 2/100
35/35 [==============================] - 1s 37ms/step - loss: 0.5254 - tp: 143.0000 - fp: 53.0000 - tn: 122.0000 - fn: 32.0000 - accuracy: 0.7571 - precision: 0.7296 - recall: 0.8171 - auc: 0.8221 - prc: 0.8206 - val_loss: 0.4805 - val_tp: 150.0000 - val_fp: 43.0000 - val_tn: 132.0000 - val_fn: 25.0000 - val_accuracy: 0.8057 - val_precision: 0.7772 - val_recall: 0.8571 - val_auc: 0.8751 - val_prc: 0.8567
Epoch 3/100
35/35 [==============================] - 1s 38ms/step - loss: 0.4367 - tp: 146.0000 - fp: 46.0000 - tn: 129.0000 - fn: 29.0000 - accuracy: 0.7857 - precision: 0

In [ ]:
result = model.evaluate(X_test_c[:500],  y_test[:500], verbose=2, batch_size=4)

In [ ]:
test_predictions = (model.predict(X_test, batch_size=4)>0.5).reshape(1,-1)[0].astype(int)
test_real_values = y_test.T[0]